# HCA Spreadsheet - SCEA MAGE-TAB Converter

## Submission information

Input here the accession number for the submission, the curators' initials, and the work directory where the csv files are located.

In [1]:
accession_index = 11
curators = ["AD", "JFG"]
work_dir = "E-HCAD-14_GSE131685"

force_project_uuid = None

# HumanColonicMesenchymeIBD
# force_project_uuid = "f8aa201c-4ff1-45a4-890e-840d63459ca2"

# KidneySingleCellAtlas
# force_project_uuid = "abe1a013-af7a-45ed-8c26-f3793c24a1f4"

# Reprogrammed_Dendritic_Cells
# force_project_uuid = "116965f3-f094-4769-9d28-ae675c1b569c"

In [2]:
accession = f"E-HCAD-{accession_index}"
protocol_accession = f"HCAD{accession_index}"
idf_file_name = f"{accession}.idf.txt"
sdrf_file_name = f"{accession}.sdrf.txt"
fill_this_label = "<FILL THIS>"

## HCA Data loading

### Helper function convert_to_snakecase

Converts a label sheet to snake_case removing " - " and whitespaces.

In [3]:
import re

def convert_to_snakecase(label):
    return re.sub(r'(\s-\s)|\s', '_', label).lower()

### Helper function to load spreadsheets

Load all csv from a directory into a dict with their names in snake_case as keys.

In [4]:
import glob
from os.path import basename, splitext

def get_all_spreadsheets(work_dir):
    file_names = glob.glob(f"{work_dir}/*.csv")
    file_names = [x for x in file_names if not 'big_table.csv' in x]
    
    spreadsheets = {}
    
    for file_name in file_names:
        spreadsheets[convert_to_snakecase(splitext(basename(file_name))[0])] = file_name

    return spreadsheets

### Load all spreadsheets

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 0)

spreadsheets = get_all_spreadsheets(work_dir)

for name, file_name in spreadsheets.items():
    spreadsheets[name] = pd.read_csv(file_name, header=0, sep=";", skiprows=[0,1,2,4])

## Making the Big Table

In [15]:
big_table = None

### Merge sequence files with cell suspensions

In [16]:
big_table = spreadsheets['cell_suspension'].merge(
    spreadsheets['sequence_file'],
    how="outer",
    on="cell_suspension.biomaterial_core.biomaterial_id"
)

### Bring specimen from cell_line if there are empty specimen ids

In [17]:
def get_specimen(cell_line_id):
    return spreadsheets['cell_line'].loc[spreadsheets['cell_line']['cell_line.biomaterial_core.biomaterial_id'] == cell_line_id]['specimen_from_organism.biomaterial_core.biomaterial_id'].values[0]

if 'cell_line' in spreadsheets.keys():
    big_table['specimen_from_organism.biomaterial_core.biomaterial_id'] = big_table['specimen_from_organism.biomaterial_core.biomaterial_id'].fillna(big_table.loc[big_table['specimen_from_organism.biomaterial_core.biomaterial_id'].isna()]['cell_line.biomaterial_core.biomaterial_id'].apply(get_specimen))

### Merge specimens into big table

In [18]:
big_table = spreadsheets['specimen_from_organism'].merge(
    big_table,
    how="outer",
    on="specimen_from_organism.biomaterial_core.biomaterial_id"
)

### Merge donor organisms into big table

In [19]:
big_table = spreadsheets['donor_organism'].merge(
    big_table,
    how="outer",
    on="donor_organism.biomaterial_core.biomaterial_id"
)

### Merge library preparation and sequencing protocols into big table

In [20]:
big_table = spreadsheets['library_preparation_protocol'].merge(
    big_table,
    how="outer",
    on="library_preparation_protocol.protocol_core.protocol_id"
)

In [21]:
big_table = spreadsheets['sequencing_protocol'].merge(
    big_table,
    how="outer",
    on="sequencing_protocol.protocol_core.protocol_id"
)

In [22]:
big_table

,sequencing_protocol.protocol_core.protocol_id,sequencing_protocol.protocol_core.protocol_name,sequencing_protocol.protocol_core.protocol_description,sequencing_protocol.protocol_core.publication_doi,sequencing_protocol.protocol_core.protocols_io_doi,sequencing_protocol.protocol_core.document,sequencing_protocol.instrument_manufacturer_model.text,sequencing_protocol.instrument_manufacturer_model.ontology,sequencing_protocol.instrument_manufacturer_model.ontology_label,sequencing_protocol.local_machine_name,sequencing_protocol.paired_end,sequencing_protocol.method.text,sequencing_protocol.method.ontology,sequencing_protocol.method.ontology_label,sequencing_protocol.10x.fastq_method,sequencing_protocol.10x.fastq_method_version,sequencing_protocol.10x.pooled_channels,sequencing_protocol.10x.drop_uniformity,library_preparation_protocol.protocol_core.protocol_id,library_preparation_protocol.protocol_core.protocol_name,library_preparation_protocol.protocol_core.protocol_description,library_preparation_protocol.protocol_core.publication_doi,library_preparation_protocol.protocol_core.protocols_io_doi,library_preparation_protocol.protocol_core.document,library_preparation_protocol.cell_barcode.barcode_read,library_preparation_protocol.cell_barcode.barcode_offset,library_preparation_protocol.cell_barcode.barcode_length,library_preparation_protocol.input_nucleic_acid_molecule.text,library_preparation_protocol.input_nucleic_acid_molecule.ontology,library_preparation_protocol.input_nucleic_acid_molecule.ontology_label,library_preparation_protocol.nucleic_acid_source,library_preparation_protocol.library_construction_method.text,library_preparation_protocol.library_construction_method.ontology,library_preparation_protocol.library_construction_method.ontology_label,library_preparation_protocol.library_construction_kit.retail_name,library_preparation_protocol.library_construction_kit.catalog_number,library_preparation_protocol.library_construction_kit.manufacturer,library_preparation_protocol.library_construction_kit.lot_number,library_preparation_protocol.library_construction_kit.expiry_date,library_preparation_protocol.end_bias,library_preparation_protocol.primer,library_preparation_protocol.strand,donor_organism.biomaterial_core.biomaterial_id,donor_organism.biomaterial_core.biomaterial_name,donor_organism.biomaterial_core.biomaterial_description,donor_organism.biomaterial_core.ncbi_taxon_id,donor_organism.biomaterial_core.genotype,donor_organism.biomaterial_core.supplementary_files,donor_organism.human_specific.ethnicity.text,donor_organism.human_specific.ethnicity.ontology,donor_organism.human_specific.ethnicity.ontology_label,donor_organism.mouse_specific.strain.ontology,donor_organism.mouse_specific.strain.ontology_label,donor_organism.genus_species.text,donor_organism.genus_species.ontology,donor_organism.genus_species.ontology_label,donor_organism.sex,donor_organism.is_living,donor_organism.organism_age,donor_organism.organism_age_unit.text,donor_organism.organism_age_unit.ontology,donor_organism.organism_age_unit.ontology_label,donor_organism.development_stage.text,donor_organism.development_stage.ontology,donor_organism.development_stage.ontology_label,donor_organism.diseases.text,donor_organism.diseases.ontology,donor_organism.diseases.ontology_label,donor_organism.gestational_age_unit.ontology,donor_organism.gestational_age_unit.ontology_label,donor_organism.height,donor_organism.height_unit.text,donor_organism.height_unit.ontology,donor_organism.height_unit.ontology_label,donor_organism.weight,...,specimen_from_organism.biomaterial_core.biomaterial_description,specimen_from_organism.biomaterial_core.ncbi_taxon_id,specimen_from_organism.biomaterial_core.genotype,specimen_from_organism.biomaterial_core.supplementary_files,specimen_from_organism.biomaterial_core.biosamples_accession,specimen_from_organism.biomaterial_core.insdc_sample_accession,specimen_from_organism.biomaterial_core.HDBR_accession,specimen_from_organism.genus_speci

### Merge the two rows for each read

In [23]:
big_table_read1 = big_table.loc[big_table['sequence_file.read_index'] == 'read1']
big_table_read2 = big_table.loc[big_table['sequence_file.read_index'] == 'read2']

big_table_read2_short = big_table_read2[[
    'cell_suspension.biomaterial_core.biomaterial_id',
    'sequence_file.file_core.file_name',
    'sequence_file.read_length',
    'sequence_file.lane_index',
]]

In [24]:
big_table_joined = big_table_read1.merge(
    big_table_read2_short,
    on=['cell_suspension.biomaterial_core.biomaterial_id', 'sequence_file.lane_index'],
    suffixes=("_read1", "_read2")
)

### Merge index rows for each read

In [25]:
big_table = big_table.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [26]:
big_table_index1 = big_table.loc[big_table['sequence_file.read_index'] == 'index1']

In [27]:
big_table_index1_short = big_table_index1[[
    'cell_suspension.biomaterial_core.biomaterial_id',
    'sequence_file.file_core.file_name',
    'sequence_file.read_length',
    'sequence_file.lane_index',
]]

big_table_index1_short.columns = [f"{x}_index1" for x in big_table_index1_short.columns]

In [28]:
big_table_joined2 = big_table_joined.merge(
    big_table_index1_short,
    left_on=['cell_suspension.biomaterial_core.biomaterial_id', 'sequence_file.lane_index'],
    right_on=["cell_suspension.biomaterial_core.biomaterial_id_index1", 'sequence_file.lane_index_index1'],
)

In [29]:
big_table_joined2.reset_index(inplace=True)
big_table_joined2 = big_table_joined2.rename(columns={'sequence_file.file_core.file_name': 'sequence_file.file_core.file_name_read1'})
big_table_joined_sorted = big_table_joined2.reindex(sorted(big_table_joined.columns), axis=1)

### Saving the Big Table

In [31]:
big_table_joined_sorted = big_table_joined_sorted.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [32]:
big_table_joined_sorted.to_csv(f"{work_dir}/big_table.csv", index=False, sep=";")

In [33]:
big_table = big_table_joined_sorted

## Mapping Protocols

In [34]:
protocol_type_map = {
    'collection_protocol': "sample collection protocol",
    'dissociation_protocol': "enrichment protocol",
    '??????????????????????': "nucleic acid extraction protocol",
    'enrichment_protocol': "enrichment_protocol",
    'library_preparation_protocol': "nucleic acid library construction protocol",
    'sequencing_protocol': "nucleic acid sequencing protocol",
}

protocol_order = [
    'collection_protocol',
    'dissociation_protocol',
    'enrichment_protocol',
    'library_preparation_protocol',
    'sequencing_protocol',
]

protocol_columns = [
    ('collection_protocol', 'collection_protocol.protocol_core.protocol_id'),
    ('library_preparation_protocol', 'library_preparation_protocol.protocol_core.protocol_id'),
    ('sequencing_protocol', 'sequencing_protocol.protocol_core.protocol_id'),
]

In [ ]:
for (protocol_type, _) in protocol_columns:
    spreadsheets[protocol_type] = spreadsheets[protocol_type].fillna('')

In [ ]:
def map_proto_to_id(protocol_name):
    for _, proto in protocol_map.items():
        if protocol_name in proto:
            return proto.get(protocol_name)['id']
    return ''

### Extract lists of protocols

Below are some helpers to convert lists in HCA spreadsheets (items are separated with two pipes `||`) to python lists.

In [ ]:
def splitlist(list_):
    split_data = []
    try:
         split_data = list_.split('||')
    except:
        pass
    
    return split_data

In [ ]:
def split_multiprotocols(df, proto_column):
    proto_series = df[proto_column].apply(splitlist)
    proto_df = pd.DataFrame(proto_series.values.tolist())
    proto_df_columns = [f'{proto_column}_{y}' for y in range(len(proto_df.columns))]
    proto_df.columns = proto_df_columns
    proto_df[f'{proto_column}_count'] = proto_series.str.len()
    proto_df[f'{proto_column}_list'] = proto_series
    
    return (proto_df, proto_df_columns)

This extracts the lists from protocol types which can have more than one instance and creates extra columns in the Big Table for each of the items, as well as the count and the python-style list.

In [ ]:
multiprotocols = [
    ('dissociation_protocol', 'dissociation_protocol.protocol_core.protocol_id'),
    ('enrichment_protocol', 'enrichment_protocol.protocol_core.protocol_id'),
]

for (protocol_type, protocol_field) in multiprotocols:
    if spreadsheets.get(protocol_type) is not None:
        spreadsheets[protocol_type] = spreadsheets[protocol_type].fillna('')
        proto_df, proto_df_columns = split_multiprotocols(big_table, protocol_field)
        for proto_column in proto_df_columns:
            protocol_columns.append( (protocol_type, proto_column) )

        big_table = big_table.merge(proto_df, left_index=True, right_index=True)

### Create protocol SCEA ids and map to HCA ids

First, we prepare an ID minter for the protocols following SCEA MAGE-TAB standards.

In [ ]:
protocol_id_counter = 0

In [ ]:
def mint_proto_id():
    global protocol_id_counter
    protocol_id_counter += 1
    return f"P-{protocol_accession}-{protocol_id_counter}"

Then, protocol map is created: a dict containing types of protocols, and inside each, a map from HCA ids to SCEA ids.

In [ ]:
protocol_map = {x: {} for x in protocol_order}

for proto_type in protocol_order:
    for (ptype, proto_column) in protocol_columns:
        if ptype == proto_type:
            new_protos = pd.unique(big_table[proto_column]).tolist()
            protocol_map[proto_type].update({proto: {
                'id': mint_proto_id()
            } for proto in new_protos if proto is not None})

Some more fields will be needed, like the description and the hardware used in some protocols, so here's a function to extract info from the protocols spreadsheets.

In [ ]:
def extract_protocol_info(column_to_extract, to_key, for_protocols = protocol_order):
    for proto_type, proto_list in protocol_map.items():
        if proto_type in for_protocols:
            for proto_name, proto in proto_list.items():
                extracted_data = spreadsheets[proto_type].loc[spreadsheets[proto_type][f'{proto_type}.protocol_core.protocol_id'] == proto_name][f'{proto_type}.{column_to_extract}'].tolist()

                if len(extracted_data):
                    proto[to_key] = extracted_data[0]
                else:
                    proto[to_key] = ''

Using that function, we get the description for all protocol types, and the hardware for sequencing protocols into the map.

In [ ]:
extract_protocol_info(f"protocol_core.protocol_description", "description")
extract_protocol_info(f"instrument_manufacturer_model.ontology_label", "hardware", ["sequencing_protocol"])

# Creating the MAGE-TAB

# IDF File

Python does not allow backslashes in f-strings, so we assign `\t` to `tab` and use that instead of a literal.

In [ ]:
tab = "\t"

Getter function for fields in spreadsheets. Will sanitize newlines into spaces.

In [ ]:
def g(sheet, col_name, func=lambda x: x):
    data = None
    
    try:
        data = tab.join(func(p) for p in list(spreadsheets[sheet][col_name].fillna(''))).replace('\n', ' ')
    except:
        pass

    return data

Getting dates from the humancellatlas API, as they are not in the spreadsheet.

In [ ]:
import requests
import json
import dateutil.parser

project_uuid = g("project", "project.uuid") if force_project_uuid is None else force_project_uuid
submission_date = fill_this_label
last_update_date = fill_this_label
geo_accessions = []

if project_uuid:
    project_url = f"https://api.ingest.data.humancellatlas.org/projects/search/findByUuid?uuid={project_uuid}"
    project_response = requests.get(project_url)
    
    if project_response.status_code == 200:
        project_data = project_response.json()

        submission_date = dateutil.parser.isoparse(project_data['submissionDate']).strftime("%Y-%m-%d")
        last_update_date = dateutil.parser.isoparse(project_data['updateDate']).strftime("%Y-%m-%d")

        geo_accessions = project_data['content'].get('geo_series_accessions', [])

In [ ]:
idf_file_contents = f"""\
MAGE-TAB Version\t1.1
Investigation Title\t{g("project", "project.project_core.project_title")}
Comment[Submitted Name]\t{g("project", "project.project_core.project_short_name")}
Experiment Description\t{g("project", "project.project_core.project_description")}
Public Release Date\t{last_update_date}
Person First Name\t{g("project_contributors", "project.contributors.name", lambda x: x.split(',')[0])}
Person Last Name\t{g("project_contributors", "project.contributors.name", lambda x: x.split(',')[2])}
Person Mid Initials\t{g("project_contributors", "project.contributors.name", lambda x: x.split(',')[1])}
Person Email\t{g("project_contributors", "project.contributors.email")}
Person Affiliation\t{g("project_contributors", "project.contributors.institution")}
Person Address\t{g("project_contributors", "project.contributors.address")}
Person Roles\t{g("project_contributors", "project.contributors.project_role.text")}
Protocol Type\t{tab.join([protocol_type_map[pt] for pt, pd in protocol_map.items() for pn, p in pd.items()])}
Protocol Name\t{tab.join([p['id'] for pt, pd in protocol_map.items() for pn, p in pd.items()])}
Protocol Description\t{tab.join([p['description'] for pt, pd in protocol_map.items() for pn, p in pd.items()])}
Protocol Hardware\t{tab.join([p.get('hardware', '') for pt, pd in protocol_map.items() for pn, p in pd.items()])}
Term Source Name\tEFO\tArrayExpress
Term Source File\thttp://www.ebi.ac.uk/efo/efo.owl\thttp://www.ebi.ac.uk/arrayexpress/
Comment[AEExperimentType]\tRNA-seq of coding RNA from single cells
Experimental Factor Name\t{fill_this_label}
Experimental Factor Type\t{fill_this_label}
Comment[EAAdditionalAttributes]\t{fill_this_label}
Comment[EACurator]\t{tab.join(curators)}
Comment[EAExpectedClusters]\t
Comment[ExpressionAtlasAccession]\t{accession}
Comment[HCALastUpdateDate]\t{last_update_date}
Comment[SecondaryAccession]\t{project_uuid}\t{tab.join(geo_accessions)}
Comment[EAExperimentType]\t{fill_this_label}
SDRF File\t{sdrf_file_name}
"""

In [ ]:
with open(f"{work_dir}/{idf_file_name}", "w") as idf_file:
    idf_file.write(idf_file_contents)

# SDRF File

## HCA - SCEA column map

Something has to be done for the case where a column is not there, spreadsheets are very random on that regard.

In [ ]:
special_columns = ['GENERIC_PROTOCOL_FIELD']
big_table['UNDEFINED_FIELD'] = fill_this_label

In [ ]:
convert_map_chunks = [{
    'Source Name': "specimen_from_organism.biomaterial_core.biomaterial_id",
    'Characteristics[organism]': "donor_organism.genus_species.ontology_label",
    'Characteristics[individual]': "donor_organism.biomaterial_core.biomaterial_id",
    'Characteristics[sex]': "donor_organism.sex",
    'Characteristics[age]': "donor_organism.organism_age",
    'Unit [time unit]': "donor_organism.organism_age_unit.text",
    'Characteristics[developmental stage]': "donor_organism.development_stage.text",
    'Characteristics[organism part]': "specimen_from_organism.organ.ontology_label",
    'Characteristics[sampling site]': "specimen_from_organism.organ_parts.ontology_label",
    'Characteristics[cell type]': "cell_suspension.selected_cell_types.ontology_label",
    'Characteristics[disease]': "donor_organism.diseases.ontology_label",
    'Characteristics[organism status]': "donor_organism.is_living",
#     'Characteristics[cause of death]': "donor_organism.death.cause_of_death",
    'Characteristics[clinical history]': "donor_organism.medical_history.test_results",
    'Comment[HCA bundle uuid]': "UNDEFINED_FIELD",
    'Comment[HCA bundle version]': "UNDEFINED_FIELD",
    'Comment[Sample_description]': "specimen_from_organism.biomaterial_core.biomaterial_description",
    'Comment[biomaterial name]': "specimen_from_organism.biomaterial_core.biomaterial_id",
    'Material Type': "UNDEFINED_FIELD",
}, {
    'Protocol REF': "GENERIC_PROTOCOL_FIELD",
}, {
    'Extract Name': "specimen_from_organism.biomaterial_core.biomaterial_id",
    'Material Type': "UNDEFINED_FIELD",
    'Comment[library construction]': "library_preparation_protocol.library_construction_method.ontology_label",
    'Comment[input molecule]': "library_preparation_protocol.input_nucleic_acid_molecule.ontology_label",
    'Comment[primer]': "library_preparation_protocol.primer",
    'Comment[end bias]': "library_preparation_protocol.end_bias",
    'Comment[umi barcode read]': "library_preparation_protocol.umi_barcode.barcode_read",
    'Comment[umi barcode offset]': "library_preparation_protocol.umi_barcode.barcode_offset",
    'Comment[umi barcode size]': "library_preparation_protocol.umi_barcode.barcode_length",
    'Comment[cell barcode read]': "library_preparation_protocol.cell_barcode.barcode_read",
    'Comment[cell barcode offset]': "library_preparation_protocol.cell_barcode.barcode_offset",
    'Comment[cell barcode size]",  ': "library_preparation_protocol.cell_barcode.barcode_length",
    'Comment[sample barcode read]': "UNDEFINED_FIELD",
    'Comment[sample barcode offset]': "UNDEFINED_FIELD",
    'Comment[sample barcode size]': "UNDEFINED_FIELD",
    'Comment[single cell isolation]': "UNDEFINED_FIELD",
    'Comment[cDNA read]': "UNDEFINED_FIELD",
    'Comment[cDNA read offset]': "UNDEFINED_FIELD",
    'Comment[cDNA read size]': "UNDEFINED_FIELD",
    'Comment[LIBRARY_STRAND]': "library_preparation_protocol.strand",
    'Comment[LIBRARY_LAYOUT]': "UNDEFINED_FIELD",
    'Comment[LIBRARY_SOURCE]': "UNDEFINED_FIELD",
    'Comment[LIBRARY_STRATEGY]': "UNDEFINED_FIELD",
    'Comment[LIBRARY_SELECTION]': "UNDEFINED_FIELD"
}, {
    'Protocol REF': "GENERIC_PROTOCOL_FIELD",
}, {
    'Assay Name': "specimen_from_organism.biomaterial_core.biomaterial_id",
    'Technology Type': "UNDEFINED_FIELD",
    'Scan Name': "specimen_from_organism.biomaterial_core.biomaterial_id",
    'Comment[RUN]': "specimen_from_organism.biomaterial_core.biomaterial_id",
    'Comment[read1 file]': "sequence_file.file_core.file_name_read1",
    'Comment[read2 file]': "sequence_file.file_core.file_name_read2",
    'Comment[index1 file]': "UNDEFINED_FIELD",
}]

## Chunk 1: donor info

In [ ]:
sdrf_1 = pd.DataFrame({k: big_table[v] for k, v in convert_map_chunks[0].items() if v not in special_columns})
sdrf_1 = sdrf_1.fillna('')

#### Fixes for chunk 1
1. Organism status: convert from 'is_alive' to 'status'

In [ ]:
sdrf_1['Characteristics[organism status]'] = sdrf_1['Characteristics[organism status]'].apply(lambda x: 'alive' if x.lower() in ['yes', 'y'] else 'dead')

## Chunk 2: collection/dissociation/enrichment protocols

In [ ]:
def convert_term(term, name):
    return map_proto_to_id(term)

In [ ]:
def convert_row(row):
    return row.apply(lambda x: convert_term(x, row.name))

In [ ]:
protocols_for_sdrf_2 = ['collection_protocol', 'dissociation_protocol', 'enrichment_protocol']

sdrf_2 = big_table[[col for proto_type, col in protocol_columns if proto_type in protocols_for_sdrf_2]]
sdrf_2 = sdrf_2.apply(convert_row)
sdrf_2.columns = ["Protocol REF" for col in sdrf_2.columns]

## Chunk 3: Library prep protocol info

In [ ]:
sdrf_3 = pd.DataFrame({k: big_table[v] for k, v in convert_map_chunks[2].items() if v not in special_columns})
sdrf_3 = sdrf_3.fillna('')

#### Fixes for chunk 3:
1. Change "Read n" to "readn" in `['Comment[umi barcode read]', 'Comment[cell barcode read]']` columns
2. In column `Comment[LIBRARY_STRAND]` add `" strand"` to the contents.


In [ ]:
read_map = {'': '', 'Read 1': "read1", 'Read 2': "read2"}

sdrf_3[['Comment[umi barcode read]', 'Comment[cell barcode read]']] = sdrf_3[['Comment[umi barcode read]', 'Comment[cell barcode read]']].applymap(lambda x: read_map[x])
sdrf_3['Comment[LIBRARY_STRAND]'] = sdrf_3['Comment[LIBRARY_STRAND]'] + " strand"

## Chunk 4: Library preparation / sequencing protocol ids

In [ ]:
protocols_for_sdrf_4 = ['library_preparation_protocol', 'sequencing_protocol']

sdrf_4 = big_table[[col for proto_type, col in protocol_columns if proto_type in protocols_for_sdrf_4]]
sdrf_4 = sdrf_4.apply(convert_row)
sdrf_4.columns = ["Protocol REF" for col in sdrf_4.columns]

## Chunk 5: Sequence files

In [ ]:
sdrf_5 = pd.DataFrame({k: big_table[v] for k, v in convert_map_chunks[4].items() if v not in special_columns})

## Merge all chunks

In [ ]:
sdrf = sdrf_1.join(sdrf_2).join(sdrf_3, rsuffix="_1").join(sdrf_4, rsuffix="_1").join(sdrf_5)

In [ ]:
sdrf = sdrf.rename(columns = {'Protocol REF_1' : "Protocol REF", 'Material Type_1': "Material Type"})

## Save SDRF file

In [ ]:
sdrf.to_csv(f"{work_dir}/{sdrf_file_name}", sep="\t")